<a href="https://colab.research.google.com/github/HazelRoma5347/CCS8/blob/main/whodunnitbutpyhton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Roma, Hazel Ann

# Step 1: Install required libraries (if not already installed)
!pip install pandas openpyxl --quiet

# Step 2: Import libraries
import pandas as pd

# Step 3: Upload the dataset
from google.colab import files
uploaded = files.upload()

# Step 4: Load the messy dataset
file_name = list(uploaded.keys())[0]  # Get the uploaded file name
df = pd.read_excel(file_name)

# Step 5: Display the records
print("Raw Data")
print(df.head())

# Step 6: Cleaning the Data
# a. Drop irrelevant columns (make sure to replace "Irrelevant Column" with the actual column name)
df = df.drop(columns=["Irrelevant Column"], errors="ignore")

# b. Remove duplicates
df = df.drop_duplicates()

# c. Standardize time formatting (convert to "hh:mm AM/PM" without seconds)
def format_time(time_str):
    if pd.isna(time_str):
        return time_str  # Return NaN as is
    time_str = str(time_str).strip()  # Ensure it's a string and strip whitespace

    # Check if the time is already in "hh:mm AM/PM" format
    if 'AM' in time_str.upper() or 'PM' in time_str.upper():
        return time_str.upper()  # Return as is if it already contains AM/PM

    # Check if the time is in "hh:mm" format
    if ':' in time_str and len(time_str.split(':')) == 2:
        return time_str + ' AM'  # Append AM for consistency

    # Try to parse as 24-hour format
    try:
        dt = pd.to_datetime(time_str, format='%H:%M:%S', errors='coerce')
        if pd.isna(dt):
            # If parsing fails, try 12-hour format
            dt = pd.to_datetime(time_str, format='%I:%M %p', errors='coerce')
        return dt.strftime('%I:%M %p') if not pd.isna(dt) else time_str
    except Exception:
        return time_str  # Return original if any error occurs

# Apply the formatting function to Entry Time and Exit Time
df['Entry Time'] = df['Entry Time'].apply(format_time)
df['Exit Time'] = df['Exit Time'].apply(format_time)

# d. Fill missing values
df["Time Near Painting (min)"] = df["Time Near Painting (min)"].fillna(0)
df["Items Carried"] = df["Items Carried"].fillna("Unknown")
df["Relation to Staff"] = df["Relation to Staff"].fillna("None")

# Step 7: Analyze the Data
# a. Filter visitors who were in Gallery B during the time of the theft (10:30 AM to 11:00 AM)
suspect_data = df[
    (df["Location Visited"].str.contains("Gallery B", na=False)) &
    (df["Time Near Painting (min)"] > 0)  # Spent time near the painting
]

# b. Sort by time near the painting (descending)
suspect_data = suspect_data.sort_values(by="Time Near Painting (min)", ascending=False)

# Step 8: Display cleaned dataset and suspect list
print("\nCleaned Dataset:")
print(df.head())

print("\nPotential Suspects:")
print(suspect_data)

# Step 9: Write your conclusion as a comment
# Conclusion:
# Based on the analysis, the suspects who spent the most time near the painting in Gallery B during the time of the theft
# are listed above. Further investigation should focus on these individuals.

Saving Roma_Gallery (1).xlsx to Roma_Gallery (1).xlsx
Raw Data
  Visitor ID        Name Entry Time Exit Time Location Visited Items Carried  \
0       V020  Visitor_20   10:24:00  10:56 AM        Gallery B           NaN   
1       V042  Visitor_42   10:05:00  12:20:00        Gallery B       Unknown   
2       V048  Visitor_48   11:37:00  22:15:00        Gallery A           Bag   
3       V013  Visitor_13   09:38:00  10:34 AM     Gallery B, C      Backpack   
4       V044  Visitor_44   11:52:00  23:10:00        Gallery C           Bag   

  Relation to Staff Suspicious Behavior  Time Near Painting (min)      TIME  \
0            Friend                 Yes                       4.0  00:32:00   
1            Friend                 Yes                      13.0  02:15:00   
2    Staff Relative                  No                      20.0  10:38:00   
3    Staff Relative                  No                       7.0  00:56:00   
4               NaN                 Yes                      